In [1]:
import sys

print(sys.version)

3.8.0 (default, Nov  6 2019, 15:49:01) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

import transformers
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification, BertForTokenClassification
from transformers import BertModel, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification, BertForTokenClassification
from tqdm import tqdm, trange

from keras.preprocessing.sequence import pad_sequences

from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [3]:
import csv

In [4]:
ner_list = []

with open("train.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")
    for line in tsvreader:
        ner_list.append(line)

In [5]:
del ner_list[131300:]
data = pd.DataFrame(ner_list,columns = ['name', 'tag'])
data

,name,tag
0,IL,B
1,-,I
2,2,I
3,gene,I
4,expression,O
...,...,...
131295,infected,O
131296,U937,O
131297,cells,O
131298,",",O


In [6]:
data.info()
data = data.dropna()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131300 entries, 0 to 131299
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   name    127109 non-null  object
 1   tag     127109 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


,name,tag
0,IL,B
1,-,I
2,2,I
3,gene,I
4,expression,O
...,...,...
131295,infected,O
131296,U937,O
131297,cells,O
131298,",",O


In [7]:
ner_list2 = []

with open("test.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")
    for line in tsvreader:
        ner_list2.append(line)
        


In [8]:
del ner_list2[131300:]
data2 = pd.DataFrame(ner_list2,columns = ['name', 'tag'])
data2 = data2.dropna()
data2

,name,tag
0,Number,O
1,of,O
2,glucocorticoid,B
3,receptors,I
4,in,O
...,...,...
118559,CD29,O
118560,+,O
118561,",",O
118562,CD45RO,O


In [9]:
data = data.dropna()
data

,name,tag
0,IL,B
1,-,I
2,2,I
3,gene,I
4,expression,O
...,...,...
131295,infected,O
131296,U937,O
131297,cells,O
131298,",",O


In [10]:
address_test = list()

for i in range(110424):
    address_test.append("1")

In [11]:
df3 = pd.DataFrame(address_test, columns = ["id"])
X_test = pd.concat([df3, data2], axis=1, join='inner')
X_test


,id,name,tag
0,1,Number,O
1,1,of,O
2,1,glucocorticoid,B
3,1,receptors,I
4,1,in,O
...,...,...,...
110419,1,sequences,O
110420,1,contains,O
110421,1,a,O
110422,1,perfect,B


In [12]:
tags_vals = list(set(X_test["tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}

In [13]:
address = list()

for i in range(126842):
    address.append("1")

In [14]:
df2 = pd.DataFrame(address, columns = ["id"])
df2

,id
0,1
1,1
2,1
3,1
4,1
...,...
126837,1
126838,1
126839,1
126840,1


In [15]:
new_data = pd.concat([df2, data], axis=1, join='inner')

In [16]:
new_data

,id,name,tag
0,1,IL,B
1,1,-,I
2,1,2,I
3,1,gene,I
4,1,expression,O
...,...,...,...
126836,1,targeting,O
126837,1,mechanism,O
126838,1,.,O
126840,1,Despite,O


In [17]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122794 entries, 0 to 126841
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      122794 non-null  object
 1   name    122794 non-null  object
 2   tag     122794 non-null  object
dtypes: object(3)
memory usage: 3.7+ MB


In [18]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w,p) for w,p in zip(s["name"].values.tolist(),
                                                        s["tag"].values.tolist())]
        self.grouped = self.dataset.groupby("id").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [19]:
getter2 = SentenceGetter(X_test)

In [20]:
getter = SentenceGetter(new_data)

In [21]:
sentences2 = [" ".join([s[0] for s in sent]) for sent in getter2.sentences]
sentences2[0]

"Number of glucocorticoid receptors in lymphocytes and their sensitivity to hormone action . The study demonstrated a decreased level of glucocorticoid receptors ( GR ) in peripheral blood lymphocytes from hypercholesterolemic subjects , and an elevated level in patients with acute myocardial infarction . In the lymphocytes with a high GR number , dexamethasone inhibited [ 3H ] - thymidine and [ 3H ] - acetate incorporation into DNA and cholesterol , respectively , in the same manner as in the control cells . On the other hand , a decreased GR number resulted in a less efficient dexamethasone inhibition of the incorporation of labeled compounds . These data showed that the sensitivity of lymphocytes to glucocorticoids changed only with a decrease of GR level . [ 1 , 25 - Dihydroxyvitamin D3 receptors in lymphocytes and T - and B - lymphocyte count in patients with glomerulonephritis ] Content of receptors to hormonal form of vitamin D3 , 1 . 25 ( OH ) 2D3 , constituted 27 . 3 fmole / m

In [22]:
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

"IL - 2 gene expression and NF - kappa B activation through CD28 requires reactive oxygen production by 5 - lipoxygenase . Activation of the CD28 surface receptor provides a major costimulatory signal for T cell activation resulting in enhanced production of interleukin - 2 ( IL - 2 ) and cell proliferation . In primary T lymphocytes we show that CD28 ligation leads to the rapid intracellular formation of reactive oxygen intermediates ( ROIs ) which are required for CD28 - mediated activation of the NF - kappa B / CD28 - responsive complex and IL - 2 expression . Delineation of the CD28 signaling cascade was found to involve protein tyrosine kinase activity , followed by the activation of phospholipase A2 and 5 - lipoxygenase . These findings should be useful for therapeutic strategies and the development of immunosuppressants targeting the CD28 costimulatory pathway . The peri - kappa B site mediates human immunodeficiency virus type 2 enhancer activation in monocytes but not in T cel

In [23]:
labels = [[s[1] for s in sent] for sent in getter.sentences]
print(labels[0])

['B', 'I', 'I', 'I', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [24]:
labels_test = [[s[1] for s in sent] for sent in getter2.sentences]
print(labels[0])

['B', 'I', 'I', 'I', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [25]:
tags_vals = list(set(new_data["tag"].values))

In [26]:
tag2idx = {t: i for i, t in enumerate(tags_vals)}

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

In [28]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [29]:
#tokenizer = tokenization.FullTokenizer(vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

In [30]:
slm = tokenizer.tokenize("Day")

In [31]:
slm

['day']

In [32]:
'''tokenized_texts = [tokenizer.tokenize(sent) for sent in data["classification"]]
tokenized_labels = [tokenizer.tokenize(sent) for sent in data["macromoleculeType"]]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
labels = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_labels]'''

'tokenized_texts = [tokenizer.tokenize(sent) for sent in data["classification"]]\ntokenized_labels = [tokenizer.tokenize(sent) for sent in data["macromoleculeType"]]\n\ninput_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]\nlabels = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_labels]'

In [33]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0])

['il', '-', '2', 'gene', 'expression', 'and', 'n', '##f', '-', 'kappa', 'b', 'activation', 'through', 'cd', '##28', 'requires', 'reactive', 'oxygen', 'production', 'by', '5', '-', 'lip', '##ox', '##y', '##genase', '.', 'activation', 'of', 'the', 'cd', '##28', 'surface', 'receptor', 'provides', 'a', 'major', 'cost', '##im', '##ulator', '##y', 'signal', 'for', 't', 'cell', 'activation', 'resulting', 'in', 'enhanced', 'production', 'of', 'inter', '##le', '##uki', '##n', '-', '2', '(', 'il', '-', '2', ')', 'and', 'cell', 'proliferation', '.', 'in', 'primary', 't', 'l', '##ym', '##ph', '##ocytes', 'we', 'show', 'that', 'cd', '##28', 'liga', '##tion', 'leads', 'to', 'the', 'rapid', 'intra', '##cellular', 'formation', 'of', 'reactive', 'oxygen', 'intermediate', '##s', '(', 'roi', '##s', ')', 'which', 'are', 'required', 'for', 'cd', '##28', '-', 'mediated', 'activation', 'of', 'the', 'n', '##f', '-', 'kappa', 'b', '/', 'cd', '##28', '-', 'responsive', 'complex', 'and', 'il', '-', '2', 'express

In [34]:
tokenized_texts_test = [tokenizer.tokenize(sent) for sent in sentences2]

In [35]:
MAX_LEN = 512

X_test = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_test],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [36]:
MAX_LEN = 512

X_train = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [37]:
er1 = tokenizer.tokenize("Covid19 ATP")
er12 = tokenizer.convert_tokens_to_ids(er1)

In [38]:
er1

['co', '##vid', '##19', 'atp']

In [39]:
er12

[2522, 17258, 16147, 12649]

In [40]:
y_train = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [41]:
y_test = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_test],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [42]:
labels

[['B',
  'I',
  'I',
  'I',
  'O',
  'O',
  'B',
  'I',
  'I',
  'I',
  'O',
  'O',
  'B',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B',
  'I',
  'I',
  'O',
  'O',
  'O',
  'O',
  'B',
  'I',
  'I',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B',
  'I',
  'I',
  'O',
  'B',
  'I',
  'I',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B',
  'I',
  'I',
  'I',
  'O',
  'B',
  'I',
  'I',
  'I',
  'O',
  'B',
  'I',
  'I',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B',
  'I',
  'I',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B',
  'I',
  'O',
  'B',
  'I',
  'I',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',

In [43]:
#X_train, X_test,y_train,y_test = train_test_split(input_ids, tags, test_size = 0.3, random_state = 42)



In [44]:
train_inputs = torch.tensor(X_train)
validation_inputs = torch.tensor(X_test)
train_labels = torch.tensor(y_train)
validation_labels = torch.tensor(y_test)

In [45]:
train_inputs

tensor([[ 6335,  1011,  1016,  4962,  3670,  1998,  1050,  2546,  1011, 16000,
          1038, 13791,  2083,  3729, 22407,  5942, 22643,  7722,  2537,  2011,
          1019,  1011,  5423, 11636,  2100, 28835,  1012, 13791,  1997,  1996,
          3729, 22407,  3302, 10769,  3640,  1037,  2350,  3465,  5714, 20350,
          2100,  4742,  2005,  1056,  3526, 13791,  4525,  1999,  9412,  2537,
          1997,  6970,  2571, 14228,  2078,  1011,  1016,  1006,  6335,  1011,
          1016,  1007,  1998,  3526, 20250,  1012,  1999,  3078,  1056,  1048,
         24335,  8458, 28788,  2057,  2265,  2008,  3729, 22407,  8018,  3508,
          5260,  2000,  1996,  5915, 26721, 16882,  4195,  1997, 22643,  7722,
          7783,  2015,  1006, 25223,  2015,  1007,  2029,  2024,  3223,  2005,
          3729, 22407,  1011, 19872, 13791,  1997,  1996,  1050,  2546,  1011,
         16000,  1038,  1013,  3729, 22407,  1011, 26651,  3375,  1998,  6335,
          1011,  1016,  3670,  1012,  3972,  3170,  

In [46]:
train_labels

tensor([[0, 2, 2, 2, 1, 1, 0, 2, 2, 2, 1, 1, 0, 1, 1, 1, 1, 1, 0, 2, 2, 1, 1, 1,
         1, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 1, 0, 2,
         2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 2, 2, 2, 1, 0, 2, 2, 2, 1,
         0, 2, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 2, 2, 1, 1, 1, 1, 1, 1,
         1, 0, 2, 1, 0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
         1, 1, 1, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 2, 0, 2, 2, 2, 1, 1, 1, 0, 2, 2, 1, 1, 0, 2, 2, 2, 2, 2, 1, 1, 1,
         1, 0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 1, 1, 1, 0, 2, 2,
         1, 1, 0, 2, 2, 2, 2

In [47]:
validation_inputs

tensor([[ 2193,  1997,  1043,  7630,  3597, 27108,  4588,  9314, 13833,  1999,
          1048, 24335,  8458, 28788,  1998,  2037, 14639,  2000, 18714,  2895,
          1012,  1996,  2817,  7645,  1037, 10548,  2504,  1997,  1043,  7630,
          3597, 27108,  4588,  9314, 13833,  1006, 24665,  1007,  1999, 15965,
          2668,  1048, 24335,  8458, 28788,  2013, 23760,  9905,  4244, 27833,
         23238,  2278,  5739,  1010,  1998,  2019,  8319,  2504,  1999,  5022,
          2007, 11325,  2026, 24755, 25070,  1999, 14971,  7542,  1012,  1999,
          1996,  1048, 24335,  8458, 28788,  2007,  1037,  2152, 24665,  2193,
          1010, 20647, 14074,  8322,  3385,  2063, 26402,  2098,  1031,  1017,
          2232,  1033,  1011, 15177,  4328, 10672,  1998,  1031,  1017,  2232,
          1033,  1011,  9078, 12259, 16935,  2046,  6064,  1998, 16480,  4244,
         27833,  1010,  4414,  1010,  1999,  1996,  2168,  5450,  2004,  1999,
          1996,  2491,  4442,  1012,  2006,  1996,  

In [48]:
validation_labels

tensor([[1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 0,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1

In [49]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [50]:
bert = AutoModel.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [51]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

In [52]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],'weight_decay_rate': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],'weight_decay_rate': 0.0}]

In [53]:
optimizer = BertAdam(optimizer_grouped_parameters,lr=2e-5,warmup=.1)

t_total value of -1 results in schedule not being applied


In [54]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [55]:
#t = [] 

# Store our loss and accuracy for plotting
#train_loss_set = []

# Number of training epochs 
#epochs = 2




In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [57]:
from sklearn.metrics import f1_score

In [58]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = BertAdam(optimizer_grouped_parameters,lr=2e-5,warmup=.1)

t_total value of -1 results in schedule not being applied


In [59]:


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [60]:
epochs = 10
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    #print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    
    

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]/Users/selimsertbas/opt/anaconda3/envs/env_tensor/lib/python3.8/site-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1005.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Train loss: 1.1616828441619873


Epoch:  10%|█         | 1/10 [00:09<01:24,  9.43s/it]

Validation loss: 0.43740314245224
Validation Accuracy: 0.94140625
Train loss: 0.7933267951011658


Epoch:  20%|██        | 2/10 [00:19<01:16,  9.55s/it]

...

Validation loss: 0.5010684728622437
Validation Accuracy: 0.94140625
Train loss: 0.5645444393157959


Epoch:  90%|█████████ | 9/10 [01:25<00:09,  9.18s/it]

Validation loss: 0.3617936372756958
Validation Accuracy: 0.94140625
Train loss: 0.5288739204406738


Epoch: 100%|██████████| 10/10 [01:35<00:00,  9.56s/it]

Validation loss: 0.35285207629203796
Validation Accuracy: 0.94140625


In [99]:
test_sentence = "glucocorticoid"

In [100]:
test_token = tokenizer.encode(test_sentence)
print(test_token)

[101, 1043, 7630, 3597, 27108, 4588, 9314, 102]


In [101]:
input_ids = torch.tensor([test_token])

In [103]:
input_ids

tensor([[  101,  1043,  7630,  3597, 27108,  4588,  9314,   102]])

In [104]:
with torch.no_grad():
    output = model(input_ids)
    

label_indices = np.argmax(output[0].to('cpu'), axis=1)

In [105]:
print(label_indices) 

tensor([1, 1, 1, 1, 1, 1, 1, 1])


In [74]:
label_indices = label_indices.numpy()
input_ids = input_ids.numpy()



In [75]:
label_indices.shape

(3,)

In [76]:
input_ids.shape

(1, 3)

In [77]:
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu'))
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[label_idx])
        new_tokens.append(token)

AttributeError: 'numpy.ndarray' object has no attribute 'to'

In [78]:
for token, label in zip(new_tokens, new_labels):
    print("{}\t{}".format(label, token))

NameError: name 'new_tokens' is not defined